# Librerías

In [ ]:
import os

# Librerias base para el analisis y manejo de datos
import numpy as np
import pandas as pd

pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Librerias con los modelos principales
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids 

# Libnreria para medir los resultados de los modelos
from sklearn.metrics import silhouette_score, adjusted_rand_score, calinski_harabasz_score

# Libreria para graficar
import matplotlib.pyplot as plt

# Estandarizar la data
from sklearn.preprocessing import StandardScaler

# Reducir la dimensionalidad para graficas
from sklearn.manifold import MDS

# metodo para recrear un iterador infinito
from itertools import cycle

# Funciones

In [ ]:
def convertir_col_numericas(df, lista_columnas):

    for columna in lista_columnas:

        df[columna] = df[columna].str.replace(',00', '', regex=False)
        df[columna] = df[columna].str.replace('.', '', regex=False)
        df[columna] = df[columna].str.replace(',', '.', regex=False).astype(float)

    return df

def porcentaje_participacion(df, lista_col, col_total):

    for col in lista_col:
        
        df[f'Porc_part_{col}'] = df[col] / df[col_total]

    return df


def crear_grafico_dispersion(modelo_fit, df, y):
    colors = plt.cm.tab20.colors + plt.cm.tab20b.colors
    color_cycle = cycle(colors)
    mds = MDS(n_components=2, dissimilarity = 'euclidean', random_state=123)
    df_transformado = mds.fit_transform(df)

    unique_labels = np.unique(modelo_fit.labels_)

    plt.figure(figsize=(10, 7))
    
    for idx, label in enumerate(unique_labels):
        color = next(color_cycle)  # Obtener el siguiente color del ciclo
        plt.scatter(df_transformado[modelo_fit.labels_ == label, 0], 
                    df_transformado[modelo_fit.labels_ == label, 1], 
                    label=f'Cluster {label}',
                    color=color)
    # Agregar las etiquetas de cada punto
    for i in range(df_transformado.shape[0]):
        plt.text(df_transformado[i, 0], 
                 df_transformado[i, 1], 
                 y.iloc[i, 0],  
                 fontsize=9, 
                 ha='right', 
                 color='black')

    plt.title('MDS Plot of Data with Labels')
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.legend()
    plt.show()

# Preparación bases de datos

## Base municipios

In [ ]:
# Base Municipios
df_base = pd.read_excel("../Data/DIVIPOLA_Municipios.xlsx")

#
display(df_base.shape)
df_base.head()

In [ ]:
df_base[df_base['CodigoM'] == 5001]

## Finanzas

In [ ]:
# Fiananzas
df_finanzas = pd.read_csv("../Data/archivos_csv/Fiananzascompilado.txt", sep="|", encoding="utf-8")

# Se toma del dataset el valor del presupuesto del SGP asignado a las entidades territoriales para el último año
df_finanzas = df_finanzas[(df_finanzas['Subcategoría']== 'SGP - Distribución asignaciones SGP por sectores') & (df_finanzas['Año']== 2023)]

# Previsualizacion de los datos
display(df_finanzas.shape)
df_finanzas.head(5)

In [ ]:
# Identificamos los indicadores que se tienen por municipio
df_finanzas['Indicador'].unique()

In [ ]:
# Filtramos aquellos datos que no son de interés para el estudio
lista_var_eliminar = ['Salud', 'Agua potable', 'Alimentación escolar', 'Primera infancia', 'Ribereños', 'Resguardos indígenas', 'Fonpet']
df_finanzas = df_finanzas[~df_finanzas['Indicador'].isin(lista_var_eliminar)]

In [ ]:
# dejar datos por municipio / son 23 años
df_finanzas1 = df_finanzas.pivot(
    index=['Código Entidad', 'Entidad'],
    columns='Indicador',
    values='Dato Numérico'
).reset_index()

In [ ]:
df_finanzas1['Educación'].dtype

In [ ]:
# Estandarizar columnas numericas
df_finanzas1 = convertir_col_numericas(df_finanzas1, ['Educación', 'Propósito general', 'Total SGP'])


In [ ]:
# Porcentaje participacion
df_finanzas1 = porcentaje_participacion(df_finanzas1, ['Educación', 'Propósito general'], 'Total SGP')

In [ ]:
df_finanzas1.head(5)

In [ ]:
# Seleccion final de variables
edu_seleccionadas = ['Código Entidad', 'Entidad', 'Porc_part_Educación', 'Porc_part_Propósito general']

df_finanzas_final = df_finanzas1[edu_seleccionadas]

In [ ]:
df_finanzas_final.head(3)

In [ ]:
df_finanzas_final.groupby('Código Entidad').size().reset_index(name='cantidad').sort_values(by='cantidad', ascending=False).head(2)


## Economia

In [ ]:
# Economia
df_economia = pd.read_csv("../Data/archivos_csv/Economia.csv", sep=";", encoding="ISO-8859-1", low_memory=False)
df_economia = df_economia[(df_economia['Año']==2023) & (df_economia['Subcategoría']== 'PIB')]
df_economia['Código Departamento'] = df_economia['Código Departamento'].astype(int)

# 
display(df_economia.shape)
df_economia.head()

In [ ]:
# Identificamos los indicadores presentes en el dataset
df_economia['Indicador'].unique()

In [ ]:
# Escogemos los indicadores relevantes para el estudio

llaves = [
    'PIB per cápita', 'PIB',
    'PIB por actividades económicas - Información y comunicaciones (Información y comunicaciones)',
    'PIB por actividades económicas - Actividades financieras (Actividades financieras y de seguros)',
    'PIB por actividades económicas - Actividades profesionales (Actividades profesionales, científicas y técnicas; actividades de servicios administrativos y de apoyo)'
]
 

lista_economia1 = ['PIB',
                  'PIB_per_capita',
                  'PIB_info_comunicaciones',
                  'PIB_act_financieras',
                  'PIB_act_prof'
                 ]

# Filtramos aquellos datos que no son de interés para el estudio
df_economia = df_economia[df_economia['Indicador'].isin(llaves)]

In [ ]:
# dejar datos por municipio
df_economia1 = df_economia.pivot(
    index=['Código Departamento', 'Departamento'],
    columns='Indicador',
    values='Dato Numérico'
).reset_index()

In [ ]:
df_economia1 = df_economia1.rename(columns={
    'PIB per cápita': 'PIB_per_capita',
    'PIB por actividades económicas - Información y comunicaciones (Información y comunicaciones)': 'PIB_info_comunicaciones',
    'PIB por actividades económicas - Actividades financieras (Actividades financieras y de seguros)': 'PIB_act_financieras',
    'PIB por actividades económicas - Actividades profesionales (Actividades profesionales, científicas y técnicas; actividades de servicios administrativos y de apoyo)': 'PIB_act_prof'
})

In [ ]:
# Estandarizar columnas numericas
df_economia1 = convertir_col_numericas(df_economia1, lista_economia1)


In [ ]:
# Porcentaje participacion
df_economia1 = porcentaje_participacion(df_economia1, lista_economia1, 'PIB')

In [ ]:
df_economia1.head(3)

In [ ]:
# Seleccion final de variables

econo_seleccionadas = ['Código Departamento', 'Departamento', 'PIB_per_capita', 'PIB_act_financieras', 'PIB_act_prof', 'PIB_info_comunicaciones']
df_economia_final = df_economia1[econo_seleccionadas]

In [ ]:
df_economia_final.head(3)

In [ ]:
df_economia_final.groupby('Departamento').size().reset_index(name='cantidad').sort_values(by='cantidad', ascending=False).head(2)


## Educacion

In [ ]:
# Economia
df_educacion = pd.read_csv("../Data/archivos_csv/Educacion.csv", sep=";", encoding="ISO-8859-1", low_memory=False)

df_educacion = df_educacion[df_educacion['Año']==2022]

# 
display(df_educacion.shape)
df_educacion.head()

In [ ]:
df_educacion['Subcategoría'].unique()

In [ ]:
lista_educacion = ['Cobertura neta en educación - Total',
                   'Cobertura neta en educación media',
                   'Tasa de repitencia del sector oficial en educación básica y media (Desde transición hasta once)']
                   

In [ ]:
df_educacion = df_educacion[df_educacion['Indicador'].isin(lista_educacion)]

In [ ]:
# dejar datos por municipio 
df_educacion1 = df_educacion.pivot(
    index=['Código Entidad', 'Entidad'],
    columns='Indicador',
    values='Dato Numérico'
).reset_index()

In [ ]:
df_educacion1 = df_educacion1.rename(columns={
    'Cobertura neta en educación - Total': 'Cobertura_edu_total',
    'Cobertura neta en educación media': 'Cobertura_edu_media',
    'Tasa de repitencia del sector oficial en educación básica y media (Desde transición hasta once)': 'Tasa_repitencia'
})

In [ ]:
lista_edu1 = ['Cobertura_edu_total', 'Cobertura_edu_media', 'Tasa_repitencia']
df_educacion1 = convertir_col_numericas(df_educacion1, lista_edu1)

In [ ]:
df_educacion1.head(5)

## Inversion en Investicacion Ciencia

In [ ]:
# Economia
df_InvCientifica = pd.read_csv("../Data/archivos_csv/Inversion_investicacion_ciencia.csv", sep=";", encoding="ISO-8859-1", low_memory=False)
df_InvCientifica = df_InvCientifica[df_InvCientifica['Año']==2021]
# 
display(df_InvCientifica.shape)
df_InvCientifica.head()

In [ ]:
df_InvCientifica['Indicador'].unique()

In [ ]:
# dejar datos por municipio / son 23 años
df_InvCientifica1 = df_InvCientifica.pivot(
    index=['Código Departamento', 'Departamento'],
    columns='Indicador',
    values='Dato Numérico'
).reset_index()

In [ ]:
df_InvCientifica1 = df_InvCientifica1.rename(columns={
    'Inversión departamental en Investigación y Desarrollo (I+D) como porcentaje del Producto Interno Bruto (PIB)': 'Inv_Ciencia'
})

In [ ]:
df_InvCientifica1 = convertir_col_numericas(df_InvCientifica1, ['Inv_Ciencia'])

In [ ]:
df_InvCientifica1.head(5)

## Mercado laboral

In [ ]:
# Economia
df_mercadoLaboral = pd.read_csv("../Data/archivos_csv/Mercado_laboral.csv", sep=";", encoding="ISO-8859-1", low_memory=False)
df_mercadoLaboral = df_mercadoLaboral[df_mercadoLaboral['Año'] == 2016]
# 
display(df_mercadoLaboral.shape)
df_mercadoLaboral.head()

In [ ]:
df_mercadoLaboral['Subcategoría'].unique()

In [ ]:
lista_laboral = ['Porcentaje de personas ocupadas formalmente con respecto a la población total']

In [ ]:
df_mercadoLaboral1 = df_mercadoLaboral[df_mercadoLaboral['Indicador'].isin(lista_laboral)]

In [ ]:
# dejar datos por municipio / son 23 años
df_mercadoLaboral1 = df_mercadoLaboral1.pivot(
    index=['Código Entidad', 'Entidad'],
    columns='Indicador',
    values='Dato Numérico'
).reset_index()

In [ ]:
df_mercadoLaboral1 = convertir_col_numericas(df_mercadoLaboral1, lista_laboral)

In [ ]:
df_mercadoLaboral1 = df_mercadoLaboral1.rename(columns={
    'Porcentaje de personas ocupadas formalmente con respecto a la población total': 'Porc_trabajos_formales'
})

In [ ]:
df_mercadoLaboral1['Porc_trabajos_formales'] = df_mercadoLaboral1['Porc_trabajos_formales'] / 100

In [ ]:
df_mercadoLaboral1.head(5)

## Transaccionalidad

In [ ]:
df_transacciones = pd.read_csv("../Data/archivos_csv/transaccionalidad.csv", sep=",", encoding="utf-8")

In [ ]:
display(df_transacciones.shape)
df_transacciones.head(3)

In [ ]:
# Seleccionamos los codigos de operacion digitales
lista_operaciones_digitales = [1, 6, 7, 8, 13, 18]
df_transacciones = df_transacciones[df_transacciones['CODIGO_CANAL_DISTRIBUCION'].isin(lista_operaciones_digitales)]

In [ ]:
df_transacciones1 = df_transacciones.groupby('CODIGO_MUNICIPIO', as_index=False)[['CANTIDAD_OPERACIONES', 'MONTO_OPERACIONES']].sum()


In [ ]:
df_transacciones1.head(5)

## Indice adopcion digital

### 2016

In [ ]:
df_CNC_2016 = pd.read_csv("../Data/archivos_csv/d2016---copia.csv", sep=";", encoding="ISO-8859-1", low_memory=False)

df_CNC_2016.head(5)

In [ ]:
col_2016 = ['PB_DANE', 'INDICADOR']
df_CNC_2016 = df_CNC_2016[col_2016]

In [ ]:
df_CNC_2016['INDICADOR'] = df_CNC_2016['INDICADOR'].str.replace(',', '.', regex=False).astype(float)

In [ ]:
df_CNC_2016.head(3)

In [ ]:
df_CNC_2016_1 = df_CNC_2016.groupby('PB_DANE', as_index=False)[['INDICADOR']].mean()
df_CNC_2016_1['Año_datos'] = 2016

In [ ]:
df_CNC_2016_1.head(3)

### 2018

In [ ]:
df_CNC_2018 = pd.read_csv("../Data/archivos_csv/d2018---copia.csv", sep=";", encoding="ISO-8859-1", low_memory=False)

df_CNC_2018.head(5)

In [ ]:
col_2018 = ['PB', 'INDICADOR']
df_CNC_2018 = df_CNC_2018[col_2018]

In [ ]:
df_CNC_2018_1 = df_CNC_2018.groupby('PB', as_index=False)[['INDICADOR']].mean()
df_CNC_2018_1['Año_datos'] = 2018

In [ ]:
df_CNC_2018_1 = df_CNC_2018_1.rename(columns={
    'PB': 'PB_DANE'
})

In [ ]:
df_CNC_2016_1.head(3)

### 2023

In [ ]:
df_CNC_2023 = pd.read_csv("../Data/archivos_csv/d2023---copia.csv", sep=";", encoding="ISO-8859-1", low_memory=False)

df_CNC_2023.head(5)

In [ ]:
col_2023 = ['DANE', 'indicador']
df_CNC_2023 = df_CNC_2023[col_2023]

In [ ]:
df_CNC_2023['indicador'] = df_CNC_2023['indicador'].str.strip()
df_CNC_2023['indicador'] = df_CNC_2023['indicador'].str.replace('-', '0', regex=False)
df_CNC_2023['indicador'] = df_CNC_2023['indicador'].str.replace(',', '.', regex=False).astype(float)

In [ ]:
df_CNC_2023_1 = df_CNC_2023.groupby('DANE', as_index=False)[['indicador']].mean()
df_CNC_2023_1['Año_datos'] = 2023

In [ ]:
df_CNC_2023_1 = df_CNC_2023_1.rename(columns={
    'DANE': 'PB_DANE',
    'indicador': 'INDICADOR'
})

In [ ]:
df_CNC_2023_1.head(3)

### Validacion columnas

In [ ]:
print(df_CNC_2016_1.columns, df_CNC_2016_1.shape[0])
print(df_CNC_2018_1.columns, df_CNC_2018_1.shape[0])
print(df_CNC_2023_1.columns, df_CNC_2023_1.shape[0])

### Union todos los datasets

In [ ]:
cnc_concat = pd.concat([df_CNC_2016_1, df_CNC_2018_1, df_CNC_2023_1], ignore_index=True)

In [ ]:
cnc_concat.shape[0]

### Seleccion valor mas actualizado por municipio

In [ ]:
# Ordena por cod_municipio y año (de más reciente a más antiguo)
cnc_concat1 = cnc_concat.sort_values(by=['PB_DANE', 'Año_datos'], ascending=[True, False])

# Elimina duplicados, conservando solo el más reciente por cod_municipio
cnc_concat1 = cnc_concat1.drop_duplicates(subset='PB_DANE', keep='first')

In [ ]:
display(cnc_concat1.shape)
cnc_concat1.head(3)

In [ ]:
cnc_concat_def = cnc_concat1[['PB_DANE', 'INDICADOR']]

# Armado dataframe Final

## Agregar info depto

In [ ]:
df1 = pd.merge(cnc_concat_def, df_base[['CodigoD','CodigoM', 'Municipio']], left_on='PB_DANE', right_on='CodigoM', how='left')

In [ ]:
df1 = df1.drop(['CodigoM'], axis=1)

In [ ]:
display(df1.shape)
df1.head(3)

## Info Finanzas

In [ ]:
df2 = pd.merge(df1, df_finanzas_final[['Código Entidad','Porc_part_Educación','Porc_part_Propósito general']], left_on='PB_DANE', right_on='Código Entidad', how='left')

In [ ]:
df2 = df2.drop(['Código Entidad'], axis=1)

In [ ]:
display(df2.shape)
df2.head(3)

## Info economía

In [ ]:
df3 = pd.merge(df2, df_economia_final[['Código Departamento','PIB_per_capita','PIB_act_financieras','PIB_act_prof','PIB_info_comunicaciones']], left_on='CodigoD', right_on='Código Departamento', how='left')

In [ ]:
df3 = df3.drop(['Código Departamento'], axis=1)
df3 = df3.fillna(0)

In [ ]:
display(df3.shape)
df3.head(3)

## Info educación

In [ ]:
df4 = pd.merge(df3, df_educacion1[['Código Entidad','Cobertura_edu_total','Cobertura_edu_media','Tasa_repitencia']], left_on='PB_DANE', right_on='Código Entidad', how='left')

In [ ]:
df4 = df4.drop(['Código Entidad'], axis=1)
#df3 = df3.fillna(0)

In [ ]:
display(df4.shape)
df4.head(3)

## Info Inv ciencia

In [ ]:
df5 = pd.merge(df4, df_InvCientifica1[['Código Departamento','Inv_Ciencia']], left_on='CodigoD', right_on='Código Departamento', how='left')

In [ ]:
df5 = df5.drop(['Código Departamento'], axis=1)
df5 = df5.fillna(0)

In [ ]:
display(df5.shape)
df5.head(3)

## Info Mercado Laboral

In [ ]:
df6 = pd.merge(df5, df_mercadoLaboral1[['Código Entidad','Porc_trabajos_formales']], left_on='PB_DANE', right_on='Código Entidad', how='left')

In [ ]:
df6 = df6.drop(['Código Entidad'], axis=1)
#df6 = df6.fillna(0)

In [ ]:
display(df6.shape)
df6.head(3)

## Info Transaccionalidad

In [ ]:
df7 = pd.merge(df6, df_transacciones1, left_on='PB_DANE', right_on='CODIGO_MUNICIPIO', how='left')

In [ ]:
df7 = df7.drop(['CODIGO_MUNICIPIO'], axis=1)

In [ ]:
display(df7.shape)
df7.head(3)

# Construccion Modelo No Supervisado

## Escalado de los datos

In [ ]:
# Seleccion variables
X = df7.drop(['PB_DANE', 'CodigoD', 'Municipio'], axis=1)
y = df7['PB_DANE']

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
df_X = pd.DataFrame(X_scaled, columns=X.columns)
df_y = pd.DataFrame(y, columns=["PB_DANE"])

In [ ]:
df = df_y.join(df_X)
df.head()

## Desarrollo modelo

### K-Means

In [ ]:
kmeans_1 = KMeans(n_clusters = 5, random_state = 42, n_init=10).fit(df_X)

In [ ]:
# Obtener las etiquetas de cluster generadas por KMeans
cluster_labels_1= kmeans_1.labels_

In [ ]:
crear_grafico_dispersion(kmeans_1, df_X, df_y)

In [ ]:
print("Metrica de silhouette_score: ",silhouette_score(df_X, kmeans_1.labels_))
print("Metrica de Calinski-Harabasz Index: ", calinski_harabasz_score(df_X, kmeans_1.labels_))
print("Metrica de Inercia de k-means: ",kmeans_1.inertia_)

### K-Medoides

In [ ]:
Kmedoides_1 = KMedoids(n_clusters = 5, random_state = 42).fit(df_X)

In [ ]:
cluster_labels_2= Kmedoides_1.labels_

In [ ]:
crear_grafico_dispersion(Kmedoides_1, df_X, df_y)

In [ ]:
print("Metrica de silhouette_score: ",silhouette_score(df_X, Kmedoides_1.labels_))
print("Metrica de Calinski-Harabasz Index: ", calinski_harabasz_score(df_X, Kmedoides_1.labels_))
print("Metrica de Inercia de k-means: ",Kmedoides_1.inertia_)

## Calibracion parámetros

### K-Means

In [ ]:
# Numero de cluster a validar
rg=50

# Inicializar estructuras necesarias para la seleccion de cluster optimo
varianza_intra_cluster = []
silhouettes = []

# iterar entre todos los posibles clusters seleccionados para encontrar el optimo
for k in range (1, rg): # Evaluamos entre 1 a rg clusters posibles
    kmeans = KMeans(n_clusters = k, random_state = 42, n_init=50).fit(df_X)
    varianza_intra_cluster.append(kmeans.inertia_)
    try:
        silhouette = silhouette_score(df_X, kmeans.labels_)
    except:
        silhouette = 0 # El índice de Silhouette sólo se puede calcular cuando se tiene K>1
    silhouettes.append(silhouette)

In [ ]:
# Graficar la varianza intra cluster por cada uno de las posibles combinaciones de cluster a validar
plt.plot(range(1, rg), varianza_intra_cluster, marker='o')
plt.xlabel('Número de clústeres (K)')
plt.ylabel('Varianza intra clúster')
plt.show()

In [ ]:
# Graficar el indice de sihoutette por cada uno de las posibles combinaciones de cluster a validar
plt.plot(range(1, rg), silhouettes, marker='o')
plt.xlabel('Número de clústeres (K)')
plt.ylabel('Índice de Silhouette')
plt.grid()
plt.show()

### K-Medoides

In [ ]:
# Numero de cluster a validar
rg=50

# Inicializar estructuras necesarias para la seleccion de cluster optimo
varianza_intra_cluster = []
silhouettes = []

# iterar entre todos los posibles clusters seleccionados para encontrar el optimo
for k in range (1, rg): # Evaluamos entre 1 a rg clusters posibles
    kmedoids = KMedoids(n_clusters = k, random_state = 42).fit(df_X)
    varianza_intra_cluster.append(kmedoids.inertia_)
    try:
        silhouette = silhouette_score(df_X, kmedoids.labels_)
    except:
        silhouette = 0 # El índice de Silhouette sólo se puede calcular cuando se tiene K>1
    silhouettes.append(silhouette)

In [ ]:
# Graficar la varianza intra cluster por cada uno de las posibles combinaciones de cluster a validar
plt.plot(range(1, rg), varianza_intra_cluster, marker='o')
plt.xlabel('Número de clústeres (K)')
plt.ylabel('Varianza intra clúster')
plt.show()

In [ ]:
# Graficar el indice de sihoutette por cada uno de las posibles combinaciones de cluster a validar
plt.plot(range(1, rg), silhouettes, marker='o')
plt.xlabel('Número de clústeres (K)')
plt.ylabel('Índice de Silhouette')
plt.grid()
plt.show()

## Mejor modelo

In [ ]:
kmeans_2 = KMeans(n_clusters = 6, random_state = 42, n_init=50).fit(df_X)

In [ ]:
crear_grafico_dispersion(kmeans_2, df_X, df_y)

In [ ]:
print("Metrica de silhouette_score: ",silhouette_score(df_X, kmeans_2.labels_))
print("Metrica de Calinski-Harabasz Index: ", calinski_harabasz_score(df_X, kmeans_2.labels_))
print("Metrica de Inercia de k-means: ",kmeans_2.inertia_)

# Analisis Clusters - Mejor Modelo

## Analisis - clusters a data set original

In [ ]:
# Obtener las etiquetas de cluster generadas por KMeans
X_clusters = df7
X_clusters.drop(columns=['PB_DANE', 'CodigoD'], inplace=True)
X_clusters.insert(0, 'clusters', kmeans_2.labels_)

# ver df con labels de clusters
X_clusters.head()

In [ ]:
# Revisar los municipios con mayor indice en que grupo quedaron
X_clusters.sort_values(by='INDICADOR', ascending=False).head(20)

In [ ]:
# describe de los grupos
X_clusters.groupby('clusters').describe()

## Conclusion - Clusters

Se identifican 6 clusters de los cuales se resaltan el cluster 3 con un solo municipio el cual es el distrito capital Bogota el cual no sera tomando en cuenta ya que su desarrollo y la inclusion digital que este presenta tiene varios factores especiales con respecto a los demas.

El cluster 5 conformado por 10 municipios los cuales presentan un indicador promedio de 0.33 y un particular porcentaje de participacion en educacion muy bajo siendo de 0.04 en promedio.

y por ultimo el cluster 1 con 22 municipios el cual presenta un indicador promedio de 0.28 y un porcentaje de participacion de educacion mucho mejor el cual es de 0.59 en promedio.

con estos dos clusters podemos evidenciar las 2 situaciones contrarias en el pais donde el indicador ya se encuentra posicionado y con los correctos planes o iniciativas en cada uno podemos mejorar la integracion con las comunidades teniendo en cuenta las necesidades y el contexto donde se encuentran los 2 clusters.

## Exportar DataSet Final

In [ ]:
X_clusters.to_csv('DataSetFinalProyectoGradoMIAD.csv', index=False)